In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from DDPG import DDPGagent
#from utils_aaa import *
import traci
import traci.constants as tc
import math
import torch
import torch.autograd
import torch.optim as optim
import torch.nn as nn
from collections import deque
import random
from model_01 import *
from utils_aa import *
from memory import *

In [2]:
import os, sys
tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
sys.path.append(tools)

In [3]:
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import pickle
filename = 'finalized_230_mode0_01_new.sav'
loaded_model_01 = pickle.load(open(filename, 'rb'))

In [5]:
import random
random.seed()
r=36.61
rl1='autonomous_vehicle'
num_veh=42
speed_total=np.zeros(42)
pos_total=np.zeros(42)
state_total_01=np.zeros(10)
action_total_01=np.zeros(2)

for episode in range(1):
#    state = env.reset()
#traci.start(["sumo-gui", "-c", "/Users/yuewang/document/maps280/ring_example_20200319-1819061584627546.990043.sumo.cfg"])
    traci.start(["sumo-gui", "-c", "/Users/yuewang/document/maps_multi-lane-ring/ring_example_20190912-1246031568277963.025501.sumo.cfg"])
    traci.simulationStep()
    vehID=traci.vehicle.getIDList()

    depart=None
    speed0="0"
    lane="0"
    typeID="human"
    routeID="routebottom"
    for i in range(7):
        str1=["v",str(i),"_0"]
        vehID="".join(str1)
        departPos_e=max(8.0*((i))+random.uniform(-1,1)*0.2,0)
        departPos=str(departPos_e)
        traci.vehicle.add(vehID, routeID, typeID, depart, lane, departPos, speed0)

    routeID="routeright"
    for i in range(7,14):
        str1=["v",str(i),"_0"]
        vehID="".join(str1)
        departPos_e=max(8.0*((i-7))+random.uniform(-1,1)*0.2,0)
        departPos=str(departPos_e)
        traci.vehicle.add(vehID, routeID, typeID, depart, lane, departPos, speed0)

    routeID="routetop"
    for i in range(14,21):
        str1=["v",str(i),"_0"]
        vehID="".join(str1)
        departPos_e=max(8.0*((i-14))+random.uniform(-1,1)*0.2,0)
        departPos=str(departPos_e)
        traci.vehicle.add(vehID, routeID, typeID, depart, lane, departPos, speed0)

    lane="1"
    typeID="human"
    routeID="routebottom"
    for i in range(7):
        str1=["v",str(i),"_1"]
        vehID="".join(str1)
        departPos_e=max(8.0*((i))+random.uniform(-1,1)*0.2,0)
        departPos=str(departPos_e)
        traci.vehicle.add(vehID, routeID, typeID, depart, lane, departPos, speed0)

    routeID="routeright"
    for i in range(7,14):
        str1=["v",str(i),"_1"]
        vehID="".join(str1)
        departPos_e=max(8.0*((i-7))+random.uniform(-1,1)*0.2,0)
        departPos=str(departPos_e)
        traci.vehicle.add(vehID, routeID, typeID, depart, lane, departPos, speed0)

    routeID="routetop"
    for i in range(14,20):
        str1=["v",str(i),"_1"]
        vehID="".join(str1)
        departPos_e=max(8.0*((i-14))+random.uniform(-1,1)*0.2,0)
        departPos=str(departPos_e)
        traci.vehicle.add(vehID, routeID, typeID, depart, lane, departPos, speed0)

    typeID="rl1"
    departPos_e=max(9.0*((20-14))+random.uniform(-1,1)*0.2,0)
    departPos=str(departPos_e)
    traci.vehicle.add("autonomous_vehicle", routeID, typeID, depart, lane, departPos, speed0)
    traci.simulationStep()
    vehID=traci.vehicle.getIDList()

    for t in range(len(vehID)):
        traci.vehicle.setSpeedMode(vehID[t],31)
        traci.vehicle.setLaneChangeMode(vehID[t],512)



# reset state
    for reset_state in range(1):
        traci.simulationStep()
        for j in range(len(vehID)):
            testVname=vehID[j]
 
            vehposition=traci.vehicle.getPosition(testVname)
            vehpos_1=vehposition[0]-r
            vehpos_2=vehposition[1]-r
        
            if vehpos_1>0 and vehpos_2<0:
                traci.vehicle.setRouteID(testVname, 'routebottom')
    
            if vehpos_1>0 and vehpos_2>0:
                traci.vehicle.setRouteID(testVname, 'routeright')
    
            if vehpos_1<0 and vehpos_2>0:
                traci.vehicle.setRouteID(testVname, 'routetop')
    
            if vehpos_1<0 and vehpos_2<0:
                traci.vehicle.setRouteID(testVname, 'routeleft')

            traci.vehicle.subscribe(testVname,(tc.VAR_LANE_INDEX,tc.VAR_SPEED,tc.VAR_POSITION))
    

    vehicle_obs_rl1=traci.vehicle.getSubscriptionResults(vehID[0])
    lane_rl1=vehicle_obs_rl1[tc.VAR_LANE_INDEX]
    veh_len=len(vehID)
    #set the line
    vehposition=vehicle_obs_rl1[tc.VAR_POSITION]
    vehpos_1=vehposition[0]-r
    vehpos_2=vehposition[1]-r

    slope=vehpos_2/vehpos_1

    theta_total=1000*np.ones(veh_len)
    theta_total_01=1000*np.ones(veh_len)

    for i in range(1,len(vehID)):
        vehicle_obs_e=traci.vehicle.getSubscriptionResults(vehID[i])
        lane_e=vehicle_obs_e[tc.VAR_LANE_INDEX]
        vehposition_e=vehicle_obs_e[tc.VAR_POSITION]
        vehpos_1_e=vehposition_e[0]-r
        vehpos_2_e=vehposition_e[1]-r
        ind_e=vehpos_2_e-slope*vehpos_1_e
    
        if slope>0 and vehpos_1 !=0 and vehpos_1>0:
            if ind_e<=0 and lane_e != lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total[i]=theta
            if ind_e<=0 and lane_e == lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total_01[i]=theta
 
    
        if slope>0 and vehpos_1 !=0 and vehpos_1<0:
            if ind_e>=0 and lane_e != lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total[i]=theta
            if ind_e<=0 and lane_e == lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total_01[i]=theta
    
        if slope<0 and vehpos_1 !=0 and vehpos_1>0:
            if ind_e<=0 and lane_e != lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total[i]=theta
            if ind_e<=0 and lane_e == lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total_01[i]=theta                

    
        if slope<0 and vehpos_1 !=0 and vehpos_1<0:
            if ind_e>=0 and lane_e != lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total[i]=theta
            if ind_e<=0 and lane_e == lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total_01[i]=theta
                
    
        if slope==0 and vehpos_1>0:
            if vehpos_2_e<=0 and lane_e != lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total[i]=theta
            if ind_e<=0 and lane_e == lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total_01[i]=theta
                
    
        if slope==0 and vehpos_1<0:
            if vehpos_2_e>=0 and lane_e != lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total[i]=theta
            if ind_e<=0 and lane_e == lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total_01[i]=theta
    
        if vehpos_1==0 and vehpos_2>0:
            if vehpos_1_e>=0 and lane_e != lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total[i]=theta
            if ind_e<=0 and lane_e == lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total_01[i]=theta
                
    
        if vehpos_1==0 and vehpos_2<0:
            if vehpos_1_e<=0 and lane_e != lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total[i]=theta
            if ind_e<=0 and lane_e == lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total_01[i]=theta


    y=np.where(theta_total==min(theta_total))
    veh_nextBehind=vehID[int(y[0])]
    vehicle_obs_e=traci.vehicle.getSubscriptionResults(veh_nextBehind)
    speed_nextBehind=vehicle_obs_e[tc.VAR_SPEED]
    
    y_01=np.where(theta_total_01==min(theta_total_01))
    veh_Behind=vehID[int(y_01[0])]
    vehicle_obs_e=traci.vehicle.getSubscriptionResults(veh_Behind)
    speed_Behind=vehicle_obs_e[tc.VAR_SPEED]
    dist_Behind=(min(theta_total_01))*r

    headway=traci.vehicle.getLeader(veh_nextBehind,200)
    headway1=headway[0]
    headway2=headway[1]

    dist=headway2
    header_rl1=headway1

    vehicle_header=traci.vehicle.getSubscriptionResults(header_rl1)
    veh_header=vehicle_header[tc.VAR_SPEED]

    veh=vehicle_obs_rl1[tc.VAR_SPEED]
    currentLaneIndex=lane_rl1

    diff_dist=(min(theta_total))*r
    headway_nextBehind=dist
    speed_nextLeader=veh_header

    state=np.array([veh,veh_header,dist,currentLaneIndex,speed_Behind,dist_Behind,speed_nextBehind,speed_nextLeader,headway_nextBehind,diff_dist])

    episode_reward = 0
    last_lc=0
    
    traci.vehicle.setSpeedMode("autonomous_vehicle",0)
#start training
    for step in range(4000):      
        
        state_01 = Variable(torch.from_numpy(state).float().unsqueeze(0))
        #state_01=state
        action_01 = loaded_model_01.actor(state_01)
        #action=actor1.forward(state_01)
        action = action_01.detach().numpy()[0]
        state_total_01=np.vstack((state_total_01,state))
        action_sub=action
        action_total_01=np.vstack((action_total_01,action_sub))
        
 # accelerating
        acc=action[0]
        Speed_e=acc*0.1+state[0]
        Speed=max(Speed_e,0)
        traci.vehicle.slowDown(rl1,Speed,0.1)
# lane-changing
        duration_step=1
        current_time=traci.simulation.getTime()
        if current_time < duration_step+last_lc:
            request=0
            direction=0
        else:
            request=1
            direction=round(action[1])

        this_lane=currentLaneIndex
        target_lane=min(max(this_lane+direction,0),1)
        traci.vehicle.changeLane(rl1,int(target_lane),1)
        traci.simulationStep()
        
# done
        pos_e=np.zeros(42)
        for j in range(len(vehID)):
            testVname=vehID[j]
            vehposition=traci.vehicle.getPosition(testVname)
            vehpos_1=vehposition[0]-r
            vehpos_2=vehposition[1]-r

            if vehpos_1>0 and vehpos_2<0:
                traci.vehicle.setRouteID(testVname, 'routebottom')
                dist_pos_e=traci.vehicle.getLanePosition(testVname)
                dist_pos=dist_pos_e
            if vehpos_1>0 and vehpos_2>0:
                traci.vehicle.setRouteID(testVname, 'routeright')
                dist_pos_e=traci.vehicle.getLanePosition(testVname)
                dist_pos=dist_pos_e+230/4
            if vehpos_1<0 and vehpos_2>0:
                traci.vehicle.setRouteID(testVname, 'routetop')
                dist_pos_e=traci.vehicle.getLanePosition(testVname)
                dist_pos=dist_pos_e+230/4*2
            if vehpos_1<0 and vehpos_2<0:
                traci.vehicle.setRouteID(testVname, 'routeleft')
                dist_pos_e=traci.vehicle.getLanePosition(testVname)
                dist_pos=dist_pos_e+230/4*3
            
            traci.vehicle.subscribe(testVname,(tc.VAR_LANE_INDEX,tc.VAR_SPEED,tc.VAR_POSITION,tc.VAR_LANEPOSITION))
        
            pos_e[j]=dist_pos
            
        pos_total=np.vstack((pos_total,pos_e))
        vehID=traci.vehicle.getIDList()

# new_state and reward
        speed_total_e=[]
        done=0           
        if done==0:
            vehicle_obs_rl1=traci.vehicle.getSubscriptionResults("autonomous_vehicle")
            
            lane_rl1=vehicle_obs_rl1[tc.VAR_LANE_INDEX]
            veh=vehicle_obs_rl1[tc.VAR_SPEED]
            veh_len=len(vehID)
            #set the line
            vehposition=vehicle_obs_rl1[tc.VAR_POSITION]
            vehpos_1=vehposition[0]-r
            vehpos_2=vehposition[1]-r
            slope=vehpos_2/vehpos_1

            theta_total=1000*np.ones(veh_len)
            theta_total_01=1000*np.ones(veh_len)
            speed_total_e=veh

            for i in range(1,len(vehID)):
                vehicle_obs_e=traci.vehicle.getSubscriptionResults(vehID[i])
                lane_e=vehicle_obs_e[tc.VAR_LANE_INDEX]
                veh_e=vehicle_obs_e[tc.VAR_SPEED]
                speed_total_e=np.hstack((speed_total_e,veh_e))

                vehposition_e=vehicle_obs_e[tc.VAR_POSITION]
                vehpos_1_e=vehposition_e[0]-r
                vehpos_2_e=vehposition_e[1]-r
                ind_e=vehpos_2_e-slope*vehpos_1_e
    
                if slope>0 and vehpos_1 !=0 and vehpos_1>0:
                    if ind_e<=0 and lane_e != lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total[i]=theta
                    if ind_e<=0 and lane_e == lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total_01[i]=theta
   
                if slope>0 and vehpos_1 !=0 and vehpos_1<0:
                    if ind_e>=0 and lane_e != lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total[i]=theta
                    if ind_e>=0 and lane_e == lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total_01[i]=theta
    
                if slope<0 and vehpos_1 !=0 and vehpos_1>0:
                    if ind_e<=0 and lane_e != lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total[i]=theta
                    if ind_e<=0 and lane_e == lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total_01[i]=theta
    
                if slope<0 and vehpos_1!=0 and vehpos_1<0:
                    if ind_e>=0 and lane_e != lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total[i]=theta
                    if ind_e>=0 and lane_e == lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total_01[i]=theta
    
                if slope==0 and vehpos_1>0:
                    if vehpos_2_e<=0 and lane_e != lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total[i]=theta
                    if vehpos_2_e<=0 and lane_e == lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total_01[i]=theta
                        
    
                if slope==0 and vehpos_1<0:
                    if vehpos_2_e>=0 and lane_e != lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total[i]=theta
                    if vehpos_2_e>=0 and lane_e == lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total_01[i]=theta
                        
    
                if vehpos_1==0 and vehpos_2>0:
                    if vehpos_1_e>=0 and lane_e != lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total[i]=theta
                    if vehpos_1_e>=0 and lane_e == lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total_01[i]=theta
    
                if vehpos_1==0 and vehpos_2<0:
                    if vehpos_1_e<=0 and lane_e != lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total[i]=theta
                    if vehpos_1_e<=0 and lane_e == lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total_01[i]=theta

                        
            y=np.where(theta_total==min(theta_total))
            veh_nextBehind=vehID[int(y[0])]
            vehicle_obs_e=traci.vehicle.getSubscriptionResults(veh_nextBehind)
            speed_nextBehind=vehicle_obs_e[tc.VAR_SPEED]

            y_01=np.where(theta_total_01==min(theta_total_01))
            veh_Behind=vehID[int(y_01[0])]
            vehicle_obs_e=traci.vehicle.getSubscriptionResults(veh_Behind)
            speed_Behind=vehicle_obs_e[tc.VAR_SPEED]
            dist_Behind=(min(theta_total_01))*r
            
            headway=traci.vehicle.getLeader(veh_nextBehind,200)
            headway1=headway[0]
            headway2=headway[1]
            
            vehicle_header_next=traci.vehicle.getSubscriptionResults(headway1)
            veh_header_next=vehicle_header_next[tc.VAR_SPEED]
            speed_nextLeader=veh_header_next
            
            vehicle_obs_e=traci.vehicle.getSubscriptionResults(veh_nextBehind)
            vehicle_obs_ee=traci.vehicle.getSubscriptionResults(headway1)

            vehposition_e=vehicle_obs_e[tc.VAR_POSITION]
            vehpos_1_e=vehposition_e[0]-r
            vehpos_2_e=vehposition_e[1]-r
            
            vehposition_ee=vehicle_obs_ee[tc.VAR_POSITION]
            vehpos_1_ee=vehposition_ee[0]-r
            vehpos_2_ee=vehposition_ee[1]-r
            cos_e=(vehpos_1_e*vehpos_1_ee+vehpos_2_e*vehpos_2_ee)/(((vehpos_1_e**2+vehpos_2_e**2)**0.5)*((vehpos_1_ee**2+vehpos_2_ee**2)**0.5))
            theta_e=math.acos(cos_e)
            dist=theta_e*r
            headway_nextBehind=dist
            
            headway=traci.vehicle.getLeader(rl1,200)
            headway1=headway[0]
            headway2=headway[1]            
            header_rl1=headway1

            vehicle_header=traci.vehicle.getSubscriptionResults(header_rl1)
            veh_header=vehicle_header[tc.VAR_SPEED]
            dist_header=headway2+7
            currentLaneIndex=lane_rl1
            
            if this_lane !=currentLaneIndex:
                last_lc=traci.simulation.getTime()


            diff_dist=(min(theta_total))*r #distance of AV and nextBehind
            
            new_state=np.array([veh,veh_header,dist_header,currentLaneIndex,speed_Behind,dist_Behind,speed_nextBehind,speed_nextLeader,headway_nextBehind,diff_dist])

            
        state = new_state
        speed_total=np.vstack((speed_total,speed_total_e))

    traci.close()



 Retrying in 1 seconds


In [59]:
import scipy.io as sio
import matplotlib.pyplot as plt
sio.savemat('saveddata_230_speed_pos_withcontrol_03_new.mat', {'speed_230_new': speed_total,'pos_230_new':pos_total})


In [60]:
import scipy.io as sio
import matplotlib.pyplot as plt
sio.savemat('saveddata_230_state_action_withcontrol_03_new.mat', {'state_230_new': state_total_01,'action_230_new':action_total_01})

